In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="gemma2-9b-It")

In [3]:
llm2 = ChatGroq(model="llama-3.1-8b-instant")

In [4]:
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

In [5]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import bs4

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_ = ("post-content", "post-title", "post-header")
        )
    )
)

In [7]:
documents = loader.load()

In [8]:
print(documents[0].page_content)



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

In [9]:
from IPython.display import display, Markdown

In [10]:
content = documents[0].page_content

In [11]:
display(Markdown(content[1200:4100]))

call (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.


Tool use

The agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.




Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem. In this process, LLM (1) translates the problem into “Problem PDDL”, then (2) requests a classical planner to generate a PDDL plan based on an existing “Domain PDDL”, and finally (3) translates the PDDL plan back into natural language. Essentially, the planning step is outsourced to an external tool, assuming the availability of domain-specific PDDL and a suitable planner which is common in certain robotic setups but not in many other domains.
Self-Reflection#
Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.
ReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space. The former enables LLM to interact with the environment (e.g. use Wikipedia search API), while the latter prompting 

In [12]:
chat_message = [
    SystemMessage(content="You are an expert with expertise in text summarization"),
    HumanMessage(content=f"""Provide a short and consise summary of following document:
    \n
    Text: {content}""")

]

In [13]:
llm.get_num_tokens(content)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (10413 > 1024). Running this sequence through the model will result in indexing errors


10413

In [14]:
display(Markdown(llm2(chat_message).content))

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Here is a concise and short summary of the document:

**LLM-Powered Autonomous Agents**

The concept of LLM-powered autonomous agents involves building agents with Large Language Models (LLMs) as their core controllers. These agents can learn to solve complex problems by breaking them down into smaller subgoals and using external memory and tool use capabilities. The agent system consists of three key components:

1. **Planning**: The agent breaks down complex tasks into smaller, manageable subgoals using techniques such as Chain of Thought (CoT) and Tree of Thoughts (ToT).
2. **Memory**: The agent uses external memory to store and retrieve information over extended periods, using techniques such as Maximum Inner Product Search (MIPS) and vector stores.
3. **Tool Use**: The agent uses external tools to extend its capabilities, such as calling APIs, executing code, and leveraging other LLMs.

Several proof-of-concept examples are presented, including AutoGPT, GPT-Engineer, and ChemCrow, which demonstrate the potential of LLM-powered autonomous agents in various domains. The challenges of building these agents include finite context length, long-term planning and task decomposition, and reliability of natural language interfaces.

The document also references several research papers and projects, including Chain of Thought, Tree of Thoughts, Reflexion, Algorithm Distillation, and API-Bank, which demonstrate the potential of LLM-powered autonomous agents in various applications.

### Prompt Template Text Summarization

In [15]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [16]:
genericPromptTemplate = """ 
    Write a summary of the following document:
    Document: {content}
    Translate the precise summary to {language}
"""

prompt = PromptTemplate(
    input_variables = ['content', 'language'],
    template=genericPromptTemplate
)

prompt

PromptTemplate(input_variables=['content', 'language'], template=' \n    Write a summary of the following document:\n    Document: {content}\n    Translate the precise summary to {language}\n')

In [17]:
complete_prompt = prompt.format(content=content, language="Hindi")

In [18]:
complete_prompt

' \n    Write a summary of the following document:\n    Document: \n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps,

In [19]:
llm2.get_num_tokens(complete_prompt)

10444

In [20]:
llm_chain = LLMChain(llm=llm2, prompt=prompt)
summary = llm_chain.run({"content":content, "language":"Hindi"})

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [21]:
display(Markdown(summary))

लिमिटेड लेंग्वेज मॉडल (LLM) द्वारा संचालित स्वायत्त एजेंट्स की विशेषताएं:

लिमिटेड लेंग्वेज मॉडल (LLM) का उपयोग करके एजेंट बनाना एक अद्वितीय अवधारणा है। कुछ प्रूफ-ऑफ-कॉन्सेप्ट डेमो जैसे कि AutoGPT, GPT-Engineer और BabyAGI ने प्रेरणादायक उदाहरण प्रदान किए हैं। LLM की क्षमता केवल अच्छी तरह से लिखी गई कॉपी, कहानियां, निबंध और प्रोग्रामों की स्थापना से परे है। यह एक शक्तिशाली सामान्य समस्या हलकर्ता के रूप में कार्य कर सकता है।

एजेंट सिस्टम का अवलोकन:

एजेंट सिस्टम में, LLM एजेंट की मानसिक क्षमता के रूप में कार्य करता है, जिसमें कई महत्वपूर्ण घटक हैं:

1. योजना: एजेंट बड़े प्रोजेक्ट को छोटे, प्रबंधनीय उप-उद्देश्यों में तोड़कर कार्य करता है। यह कार्य करने में कुशल होता है, विशेष रूप से जटिल कार्यों के लिए।

2. याद: LLM एजेंट को अतीत कार्यों को याद रखने और पुनः प्राप्त करने की क्षमता प्रदान करता है, जिससे यह अनंत समय तक जानकारी को संग्रहीत और पुनः प्राप्त कर सकता है।

3. टूल उपयोग: एजेंट वाह्य जानकारी को प्राप्त करने और पुनः प्राप्त करने के लिए वाह्य एपीआई का उपयोग करने में सक्षम होता है, जिससे यह जानकारी को संग्रहीत और पुनः प्राप्त कर सकता है।

प्लानिंग घटक:

एजेंट की योजना बनाने की प्रक्रिया में निम्नलिखित घटक शामिल हैं:

1. उप-उद्देश्य विभाजन: एजेंट बड़े प्रोजेक्ट को छोटे, प्रबंधनीय उप-उद्देश्यों में तोड़ता है, जिससे यह कार्य करने में कुशल होता है।

2. आत्म-समीक्षा और सुधार: एजेंट अतीत कार्यों का आत्म-समीक्षा करता है और उन्हें सुधारता है, जिससे यह कार्य करने में कुशल होता है।

मेमोरी घटक:

एजेंट की याद की क्षमता में निम्नलिखित घटक शामिल हैं:

1. अल्पावधि याद: LLM एजेंट को अतीत कार्यों को याद रखने और पुनः प्राप्त करने की क्षमता प्रदान करता है, जिससे यह अल्पावधि में जानकारी को संग्रहीत और पुनः प्राप्त कर सकता है।

2. दीर्घकालिक याद: LLM एजेंट को अनंत समय तक जानकारी को संग्रहीत और पुनः प्राप्त करने की क्षमता प्रदान करता है, जिससे यह दीर्घकालिक में जानकारी को संग्रहीत और पुनः प्राप्त कर सकता है।

टूल उपयोग घटक:

एजेंट के टूल उपयोग की क्षमता में निम्नलिखित घटक शामिल हैं:

1. वाह्य एपीआई का उपयोग: एजेंट वाह्य एपीआई का उपयोग करके अतिरिक्त जानकारी प्राप्त कर सकता है, जिससे यह जानकारी को संग्रहीत और पुनः प्राप्त कर सकता है।

एजेंट सिस्टम की प्रक्रिया:

एजेंट सिस्टम की प्रक्रिया में निम्नलिखित चरण शामिल हैं:

1. योजना: एजेंट बड़े प्रोजेक्ट को छोटे, प्रबंधनीय उप-उद्देश्यों में तोड़ता है, जिससे यह कार्य करने में कुशल होता है।

2. याद: एजेंट अतीत कार्यों को याद रखता है और उन्हें पुनः प्राप्त करता है, जिससे यह कार्य करने में कुशल होता है।

3. टूल उपयोग: एजेंट वाह्य एपीआई का उपयोग करके अतिरिक्त जानकारी प्राप्त कर सकता है, जिससे यह जानकारी को संग्रहीत और पुनः प्राप्त कर सकता है।

एजेंट के घटक:

एजेंट के कई घटक हैं, जिनमें से कुछ प्रमुख घटक निम्नलिखित हैं:

1. योजना: एजेंट बड़े प्रोजेक्ट को छोटे, प्रबंधनीय उप-उद्देश्यों में तोड़ता है, जिससे यह कार्य करने में कुशल होता है।

2. याद: एजेंट अतीत कार्यों को याद रखता है और उन्हें पुनः प्राप्त करता है, जिससे यह कार्य करने में कुशल होता है।

3. टूल उपयोग: एजेंट वाह्य एपीआई का उपयोग करके अतिरिक्त जानकारी प्राप्त कर सकता है, जिससे यह जानकारी को संग्रहीत और पुनः प्राप्त कर सकता है।

एजेंट के उदाहरण:

एजेंट के कई उदाहरण हैं, जिनमें से कुछ प्रमुख उदाहरण निम्नलिखित हैं:

1. AutoGPT: यह एक प्रूफ-ऑफ-कॉन्सेप्ट डेमो है जो एजेंट की क्षमता को प्रदर्शित करता है।

2. GPT-Engineer: यह एक अन्य प्रूफ-ऑफ-कॉन्सेप्ट डेमो है जो एजेंट की क्षमता को प्रदर्शित करता है।

3. BabyAGI: यह एक अन्य प्रूफ-ऑफ-कॉन्सेप्ट डेमो है जो एजेंट की क्षमता को प्रदर्शित करता है।

एजेंट के सीमाएं:

एजेंट के कई सीमाएं हैं, जिनमें से कुछ प्रमुख सीमाएं निम्नलिखित हैं:

1. अल्पावधि याद: एजेंट अल्पावधि में जानकारी को संग्रहीत और पुनः प्राप्त करने में असमर्थ होता है।

2. दीर्घकालिक याद: एजेंट दीर्घकालिक में जानकारी को संग्रहीत और पुनः प्राप्त करने में असमर्थ होता है।

3. टूल उपयोग: एजेंट वाह्य एपीआई का उपयोग करके अतिरिक्त जानकारी प्राप्त करने में असमर्थ होता है।

एजेंट के भविष्य की दिशा:

एजेंट के भविष्य में कई दिशाएं हैं, जिनमें से कुछ प्रमुख दिशाएं निम्नलिखित हैं:

1. अल्पावधि याद का विस्तार: एजेंट को अल्पावधि में जानकारी को संग्रहीत और पुनः प्राप्त करने में सक्षम बनाना।

2. दीर्घकालिक याद का विस्तार: एजेंट को दीर्घकालिक में जानकारी को संग्रहीत और पुनः प्राप्त करने में सक्षम बनाना।

3. टूल उपयोग का विस्तार: एजेंट को वाह्य एपीआई का उपयोग करके अतिरिक्त जानकारी प्राप्त करने में सक्षम बनाना।

एजेंट के संदर्भ:

एजेंट के कई संदर्भ हैं, जिनमें से कुछ प्रमुख संदर्भ निम्नलिखित हैं:

1. इंजीनियरिंग: एजेंट को इंजीनियरिंग में उपयोग किया जा सकता है।

2. व्यवसाय: एजेंट को व्यवसाय में उपयोग किया जा सकता है।

3. शिक्षा: एजेंट को शिक्षा में उपयोग किया जा सकता है।

एजेंट के संदर्भों के अलावा, एजेंट के कई अन्य महत्वपूर्ण पहलुओं पर विचार करना आवश्यक है, जिनमें से कुछ प्रमुख पहलू निम्नलिखित हैं:

1. एजेंट की सुरक्षा: एजेंट की सुरक्षा एक महत्वपूर्ण पहलू है, जिससे एजेंट को हानि पहुंचाने से बचाया जा सकता है।

2. एजेंट का नियंत्रण: एजेंट का नियंत्रण एक महत्वपूर्ण पहलू है, जिससे एजेंट को सही तरीके से कार्य करने में सक्षम बनाया जा सकता है।

3. एजेंट का डेटा उपयोग: एजेंट का डेटा उपयोग एक महत्वपूर्ण पहलू है, जिससे एजेंट को सही तरीके से कार्य करने में सक्षम बनाया जा सकता है।

इन पहलुओं को ध्यान में रखते हुए, एजेंट एक शक्तिशाली उपकरण है जो कई क्षेत्रों में उपयोग किया जा सकता है।

## Stuff Document Chain Text Summarization

In [22]:
!dir

1.\ Start	     Chatbot			    chroma_db
2.\ DataIngestion    LCEL\ &\ GROQ		    faiss_index
3.\ Text\ Splitting  Langchain\ and\ Azure\ OpenAI  main.py
4.\ Embedding	     LangchainAndOllama		    requirements.txt
5.\ VectorStores     Projects


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Projects/Text Summarization/uber_10q_sept_2022.pdf")

In [24]:
pages = loader.load_and_split()
len(pages)

158

In [25]:
template = """
    Write a concise and short summary of following financial report of Uber
    Report : {text}

    """
prompt = PromptTemplate(input_variables=["text"], template=template)

In [26]:
from langchain.chains.summarize import load_summarize_chain

In [27]:
chain = load_summarize_chain(llm2, chain_type='stuff', prompt=prompt, verbose=True)

In [28]:
output_summary = chain.run(pages)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Write a concise and short summary of following financial report of Uber
    Report : UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
____________________________________________ 
FORM 10-Q
____________________________________________ 
(Mark One)
☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the quarterly period ended September 30, 2022
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from_____ to _____            
Commission File Number: 001-38902
____________________________________________ 
UBER TECHNOLOGIES, INC.
(Exact name of registrant as specified in its charter)
Not Applicable
(Former name, former address and former fiscal year, if changed since last report)
_______________________________________________________________________

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j35djpdtfb5sy5d1bc5y30y6` on requests per minute (RPM): Limit 30, Used 30, Requested 1. Please try again in 1.91s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'requests', 'code': 'rate_limit_exceeded'}}

In [ ]:
display(Markdown(output_summary))

The provided text is a quarterly report on Form 10-Q for Uber Technologies, Inc. for the period ended September 30, 2022. The report includes financial statements, management's discussion and analysis, and other information related to the company's operations and financial condition.

The report highlights the company's revenue growth, driven by an increase in Gross Bookings, which rose to $29.1 billion in the third quarter of 2022. Revenue increased by 72% year-over-year, outpacing Gross Bookings growth due to a $1.3 billion increase in Freight business and a net favorable impact to Mobility revenue. Net loss attributable to Uber Technologies, Inc. was $1.2 billion, which includes a pre-tax unrealized loss on debt and equity securities.

The report also discusses the company's segment performance, with Mobility Adjusted EBITDA profit increasing by $354 million, and Delivery Adjusted EBITDA growing by $193 million. Freight Adjusted EBITDA improved by $36 million.

The report mentions various risks and uncertainties that may impact the company's business, including the COVID-19 pandemic, competition, regulatory challenges, and data security breaches. The company has made climate-related commitments, including a commitment to 100% renewable electricity for its U.S. offices by 2025, and a commitment to net zero climate emissions from corporate operations by 2030.

The report also highlights the company's exposure to market risks, including interest rate risk, investment risk, and foreign currency risk. The company's 2025 Refinanced Term Loan and 2027 Refinanced Term Loan Facilities represent floating rate notes and are carried at amortized cost. A rising interest rate environment will increase the amount of interest paid on these loans.

The report includes various financial statements, including the condensed consolidated balance sheet, condensed consolidated statements of operations, and condensed consolidated statements of comprehensive income (loss). The report also includes notes to the financial statements, which provide additional information about the company's financial position and results of operations.

The report is signed by Dara Khosrowshahi, the Chief Executive Officer and Director, and Nelson Chai, the Chief Financial Officer, who certify that the report fully complies with the requirements of Section 13(a) or 15(d) of the Securities Exchange Act of 1934 and that the information contained in the report fairly presents, in all material respects, the financial condition and results of operations of Uber Technologies, Inc.

Overall, the report provides a comprehensive overview of Uber Technologies, Inc.'s financial performance and operations for the period ended September 30, 2022.

### Map Reduce Chain Text Summarization

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
final_documents = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200).split_documents(pages)
 

In [ ]:
final_documents

[Document(metadata={'source': 'Projects/Text Summarization/uber_10q_sept_2022.pdf', 'page': 0}, page_content='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n____________________________________________ \nFORM 10-Q\n____________________________________________ \n(Mark One)\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended September 30, 2022\nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from_____ to _____            \nCommission File Number: 001-38902\n____________________________________________ \nUBER TECHNOLOGIES, INC.\n(Exact name of registrant as specified in its charter)\nNot Applicable\n(Former name, former address and former fiscal year, if changed since last report)\n____________________________________________________________________________ \nDelaware 45-2647441\n(State or other jurisdiction of incorpo

In [ ]:
final_documents[0].page_content

'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n____________________________________________ \nFORM 10-Q\n____________________________________________ \n(Mark One)\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended September 30, 2022\nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from_____ to _____            \nCommission File Number: 001-38902\n____________________________________________ \nUBER TECHNOLOGIES, INC.\n(Exact name of registrant as specified in its charter)\nNot Applicable\n(Former name, former address and former fiscal year, if changed since last report)\n____________________________________________________________________________ \nDelaware 45-2647441\n(State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.)\n1515 3rd Street\nSan Francisco, California 9415

In [ ]:
[doc.page_content for doc in final_documents]

['UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n____________________________________________ \nFORM 10-Q\n____________________________________________ \n(Mark One)\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended September 30, 2022\nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from_____ to _____            \nCommission File Number: 001-38902\n____________________________________________ \nUBER TECHNOLOGIES, INC.\n(Exact name of registrant as specified in its charter)\nNot Applicable\n(Former name, former address and former fiscal year, if changed since last report)\n____________________________________________________________________________ \nDelaware 45-2647441\n(State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.)\n1515 3rd Street\nSan Francisco, California 941

In [ ]:
len(final_documents)

336

In [ ]:
chunk_prompt = """
    Summarize below report:
    Report: `{text}`
    Summary:
"""


map_prompt_template = PromptTemplate(input_variables=['text'], template=chunk_prompt)

In [ ]:
final_prompt = """
    Provide the final summary of entire report with these import points.
    Add a suitable Title, Start the precise summary with an introduction and provide
    the summary in numbers whenever necessary.
    Points for Report
    Report:{text}
"""

final_prompt_template = PromptTemplate(input_variables=['text'], template=final_prompt)

In [ ]:
final_prompt_template

PromptTemplate(input_variables=['text'], template='\n    Provide the final summary of entire report with these import points.\n    Add a suitable Title, Start the precise summary with an introduction and provide\n    the summary in numbers whenever necessary.\n    Points for Report\n    Report:{text}\n')

In [ ]:
summary_chain = load_summarize_chain(
    llm=llm2, 
    chain_type='map_reduce', 
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template 
    )

In [ ]:
output = summary_chain.run(final_documents)

In [ ]:
display(Markdown(output))

Please provide me with the excerpt from the Uber Technologies, Inc. report that you would like summarized.  

Once you give me the text, I can create a comprehensive and accurate summary for you, incorporating the key points you've outlined. 
